In [1]:
import os
import asyncio
import logging 
from pathlib import Path
from typing import Tuple, Any, List, Dict, Optional
from dataclasses import dataclass
from dotenv import load_dotenv

from langchain_cohere import CohereEmbeddings
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import (
    TextLoader, 
    UnstructuredMarkdownLoader,
    JSONLoader,
    UnstructuredHTMLLoader,
    PyPDFLoader
)

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnableLambda
from langchain.callbacks.base import AsyncCallbackHandler
from langchain.schema import LLMResult

load_dotenv()
# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [2]:
@dataclass
class SelfRAGResponse:
    """Complete self rag with reflection"""
    answer: str
    retrieved_docs: List[Document]
    reflection_score: float
    needs_retrieval: bool
    citations: List[str]
    retrieval_decision_reasoning: str


class RateLimitCallback(AsyncCallbackHandler):
    """Callback handler to manage API rate limiting with semaphores"""
    
    def __init__(self, semaphore: asyncio.Semaphore):
        self.semaphore = semaphore
        
    async def on_llm_start(self, serialized: Dict[str, Any], prompts: List[str], **kwargs: Any) -> None:
        await self.semaphore.acquire()
        
    async def on_llm_end(self, response: LLMResult, **kwargs: Any) -> None:
        self.semaphore.release()

In [3]:
class DocumentLoader:
    def __init__(self):
        self.loaders = {
            '.txt': TextLoader,
            '.md': UnstructuredMarkdownLoader,
            '.json': self._create_json_loader,
            '.html': UnstructuredHTMLLoader,
            '.py': TextLoader,
            '.js': TextLoader,
            '.css': TextLoader,
            '.pdf': PyPDFLoader
        }

    def _create_json_loader(self, file_path: str):
        """Create JSON loader with custom jq_schema"""
        return JSONLoader(
            file_path=file_path,
            jq_schema='.[]',
            text_content=False
        )

    async def load_documents(self, kb_folder: str) -> List[Document]:
        """Load all documents from the knowledge base folder"""
        documents = []
        kb_path = Path(kb_folder)

        if not kb_path.exists():
            raise FileNotFoundError(f"Knowledge base folder not found: {kb_path}")

        for file_path in kb_path.glob("**/*"):
            if file_path.is_file() and file_path.suffix.lower() in self.loaders:
                try:
                    loader_class = self.loaders[file_path.suffix.lower()]

                    if file_path.suffix.lower() == ".json":
                        loader = loader_class(str(file_path))
                    else:
                        loader = loader_class(str(file_path))

                    docs = loader.load()

                    # Add metadata
                    for doc in docs:
                        doc.metadata.update({
                            'file_path': str(file_path),
                            'file_type': file_path.suffix,
                            'file_name': file_path.name
                        })

                    documents.extend(docs)
                
                except Exception as e:
                    logger.warning(f"There was an error loading the knowledge base: {str(e)}")
                    # Fallback to TextLoader for unknown formats
                    try:
                        loader = TextLoader(str(file_path))
                        docs = loader.load()
                        # Add metadata
                        for doc in docs:
                            doc.metadata.update({
                                'file_path': str(file_path),
                                'file_type': file_path.suffix,
                                'file_name': file_path.name
                            })

                        documents.extend(docs)

                    except Exception as fallback_error:
                        logger.error(f"Failed to load {file_path} with fallback: {fallback_error}")
        
        logger.info(f"Loaded {len(documents)} documents from {kb_folder}")
        return documents

In [7]:
from asyncio import Semaphore

@dataclass
class RAGSystem:
    cohere_api_key: str
    openrouter_api_key: str
    kb_folder: str
    vector_store_path: str = None 
    max_concurrent_requests: int = 5
    chunk_size: int = 2000
    chunk_overlap: int = 200
    auto_save_vector_store: bool = True 
    def __post_init__(self):
        # Initialize the components
        self.embeddings = CohereEmbeddings(model = "embed-v4.0",
                                         cohere_api_key = os.getenv("COHERE_API_KEY"))

        self.llm = ChatOpenAI(
            model="meta-llama/llama-3.3-70b-instruct",
            openai_api_key=self.openrouter_api_key,
            openai_api_base="https://openrouter.ai/api/v1",
            temperature=0.6,
            max_tokens=1500
        )

        # Text Splitter
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size = self.chunk_size,
            chunk_overlap = self.chunk_overlap,
            length_function = len
        )

        self.document_loader = DocumentLoader()

        # Vector store
        self.vector_store: Optional[FAISS] = None
        
        # Set default vector store path if not provided
        if self.vector_store_path is None:
            self.vector_store_path = os.path.join(self.kb_folder, "vector_store")

        # Semaphores for rate limiting
        self.llm_semaphore = Semaphore(self.max_concurrent_requests)
        self.embeddings_semaphore = Semaphore(self.max_concurrent_requests)

        self.rate_limit_callback = RateLimitCallback(self.llm_semaphore)

        self.is_initialized = False

        # Set up prompts
        self._setup_prompts()

    def _setup_prompts(self):
        """Set up prompts for different stages"""

        # Retrieval decision prompt
        self.retrieval_decision_prompt = PromptTemplate(
            input_variables=["query"],
            template="""
            Analyze the following query to determine if it requires external knowledge retrieval.
            
            Query: "{query}"
            
            Consider:
            1. Does this query ask for specific facts, data, or domain-specific information?
            2. Would the answer benefit from external documents or knowledge base?
            3. Is this asking about general knowledge that can be answered without retrieval?
            4. Does it require recent or specialized information?
            
            Provide your reasoning and then answer with either "RETRIEVE" or "NO_RETRIEVE".
            
            Reasoning: [Explain your decision]
            Decision: [RETRIEVE or NO_RETRIEVE]
            """
        )

        # Answer generation with retrieval prompt
        self.rag_prompt = ChatPromptTemplate.from_template("""
            You are a helpful AI assistant. Use the following context documents to answer the user's question accurately and comprehensively.
            
            Context Documents:
            {context}
            
            Question: {question}
            
            Instructions:
            - Base your answer primarily on the provided context
            - If the context doesn't contain sufficient information, acknowledge this
            - Cite specific documents when referencing information
            - Be accurate, detailed, and helpful
            - If you need to use general knowledge to supplement the context, clearly indicate this
            
            Answer:
        """)

        # Answer generation without retrieval prompt
        self.no_retrieval_prompt = ChatPromptTemplate.from_template("""
            You are a helpful AI assistant. Answer the following question using your general knowledge.
            
            Question: {question}
            
            Provide a comprehensive and accurate answer based on your training knowledge.
            
            Answer:
        """)

        # Reflection prompt
        self.reflection_prompt = PromptTemplate(
            input_variables=["query", "answer", "context"],
            template="""
            Evaluate the quality of the following answer based on the query and available context.
            
            Query: {query}
            
            Context:
            {context}
            
            Answer: {answer}
            
            Rate the answer on a scale of 0-10 considering:
            - Accuracy and factual correctness
            - Completeness and comprehensiveness
            - Relevance to the query
            - Proper use of available context
            - Clarity and helpfulness
            
            Provide only a single number between 0 and 10 as your rating.
            
            Rating:
            """
        )

    async def initialize(self, force_rebuild: bool = False):
        """Initialize the RAG System with option to force rebuild"""
        if self.is_initialized and not force_rebuild:
            return
        
        logger.info("Initializing the RAG system ...")

        # Try to load existing vector store first 
        if not force_rebuild and os.path.exists(self.vector_store_path):
            try:
                await self.load_vector_store(self.vector_store_path)
                logger.info("Loaded existing vector store successfully")
                return
            except Exception as e:
                logger.warning(f"Failed to load existing vector store: {e}. Building new one...")

        # Load the documents
        documents = await self.document_loader.load_documents(self.kb_folder)

        if not documents:
            logger.warning("No documents were found in the knowledge base ...")
            self.vector_store = None
            self.is_initialized = True
            return

        # Split the documents into chunks
        split_docs = self.text_splitter.split_documents(documents)
        logger.info(f"Split {len(documents)} documents into {len(split_docs)} chunks")

        # Create the vector store
        self.vector_store = await FAISS.afrom_documents(
            split_docs,
            self.embeddings
        )

        # ALWAYS SAVE THE VECTOR STORE AFTER CREATION
        if self.auto_save_vector_store:
            await self.save_vector_store(self.vector_store_path)
            logger.info(f"Vector store automatically saved to {self.vector_store_path}")

        self.is_initialized = True
        logger.info("Self-RAG system initialized successfully")

    async def _should_retrieve(self, query: str) -> Tuple[bool, str]:
        """ Determine if the retrieval is needed and get reasoning"""
        chain = self.retrieval_decision_prompt | self.llm.with_config(callbacks=[self.rate_limit_callback])

        result = await chain.ainvoke({"query": query})

        # Parse the result
        lines = result.content.strip().split('\n')
        reasoning = ""
        decision = False

        for line in lines:
            if line.startswith("Reasoning:"):
                reasoning = line.replace("Reasoning:", "").strip()
            elif line.startswith("Decision:"):
                decision_text = line.replace("Decision:", "").strip()
                decision = "RETRIEVE" in decision_text.upper()
        
        return decision, reasoning

    async def _retrieve_documents(self, query: str, k:int = 5) -> List[Document]:
        """Retrieve relevant documents"""
        if not self.vector_store:
            return []

        # Use similarity search with scores
        doc_with_scores = await self.vector_store.asimilarity_search_with_score(query, k = k)
        # Filter by relevance score
        relevant_docs = [doc for doc, score in doc_with_scores if score > 0.8]

        return relevant_docs

    async def _generate_answer_with_retrieval(self, query: str, documents: List[Document]) -> str:
        """Generate answers using retrieved documents"""
        context = "\n\n".join([
            f"Document: {doc.metadata.get('file_name', 'Unknown')}\n{doc.page_content}"
            for doc in documents
        ])
        
        chain = self.rag_prompt | self.llm | StrOutputParser()
        
        result = await chain.ainvoke({
            "context": context,
            "question": query
        })
        
        return result

    async def _generate_answer_without_retrieval(self, query: str) -> str:
        """Generate answer without retrieval"""
        chain = self.no_retrieval_prompt | self.llm | StrOutputParser()
        
        result = await chain.ainvoke({"question": query})
        return result

    async def _reflect_on_answer(self, query: str, answer: str, documents: List[Document]) -> float:
        """Reflect on answer quality"""
        context = "\n\n".join([doc.page_content for doc in documents]) if documents else "No context provided"
        
        chain = self.reflection_prompt | self.llm.with_config(callbacks=[self.rate_limit_callback])
        
        result = await chain.ainvoke({
            "query": query,
            "answer": answer,
            "context": context
        })
        
        try:
            content = result.content if hasattr(result, 'content') else str(result)
            score_str = content.strip().split('\n')[-1]
            
            # Extract digits and decimal point from the score string
            score_chars = ''.join(c for c in score_str if c.isdigit() or c == '.')
            
            if score_chars:
                score = float(score_chars)
                return min(10.0, max(0.0, score))  # Clamp between 0 and 10
            else:
                logger.warning(f"No numeric score found in: {score_str}")
                return 5.0
                
        except (ValueError, IndexError) as e:
            logger.warning(f"Could not parse reflection score: {result}. Error: {e}")
            return 5.0

    async def query(self, query: str, force_retrieval: bool = False) -> SelfRAGResponse:
        """Process query using the SelfRAG"""
        if not self.is_initialized:
            await self.initialize()

        # Decide whether we need retrieval
        if force_retrieval:
            needs_retrieval = True
            reasoning = "Forced retrieval requested"
        else:
            needs_retrieval, reasoning = await self._should_retrieve(query)

        retrieved_docs = []
        citations = []

        # Retrieve documents if needed
        if needs_retrieval and self.vector_store:
            retrieved_docs = await self._retrieve_documents(query)
            citations = [
                doc.metadata.get('file_path', f"Document {i}")
                for i, doc in enumerate(retrieved_docs)
            ]

        # Generate answer
        if retrieved_docs:
            answer = await self._generate_answer_with_retrieval(query, retrieved_docs)
        else:
            answer = await self._generate_answer_without_retrieval(query)

        # Reflect on answer quality
        reflection_score = await self._reflect_on_answer(query, answer, retrieved_docs)

        return SelfRAGResponse(
            answer=answer,
            retrieved_docs=retrieved_docs,
            reflection_score=reflection_score,
            needs_retrieval=needs_retrieval,
            citations=list(set(citations)),  # Remove duplicates
            retrieval_decision_reasoning=reasoning
        )

    async def save_vector_store(self, path: str = None):
        """Save vector store to disk"""
        if not self.vector_store:
            logger.warning("No vector store to save")
            return
            
        save_path = path or self.vector_store_path
        
        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        
        try:
            self.vector_store.save_local(save_path)
            logger.info(f"Vector store saved to {save_path}")
        except Exception as e:
            logger.error(f"Failed to save vector store: {e}")
            raise
    
    async def load_vector_store(self, path: str = None):
        """Load vector store from disk"""
        load_path = path or self.vector_store_path
        
        try:
            self.vector_store = FAISS.load_local(load_path, self.embeddings, allow_dangerous_deserialization=True)
            self.is_initialized = True
            logger.info(f"Vector store loaded from {load_path}")
        except Exception as e:
            logger.error(f"Failed to load vector store from {load_path}: {e}")
            raise

    async def rebuild_vector_store(self):
        """Force rebuild the vector store from documents"""
        logger.info("Force rebuilding vector store...")
        await self.initialize(force_rebuild=True)

In [8]:
class BatchProcessor:
    """Process multiple queries in batch with concurrency control"""
    
    def __init__(self, rag_system: RAGSystem, max_concurrent: int = 5):
        self.rag_system = rag_system
        self.semaphore = asyncio.Semaphore(max_concurrent)
    
    async def process_query(self, query: str) -> Tuple[str, SelfRAGResponse]:
        """Process a single query with semaphore"""
        async with self.semaphore:
            response = await self.rag_system.query(query)
            return query, response
    
    async def process_batch(self, queries: List[str]) -> Dict[str, SelfRAGResponse]:
        """Process multiple queries concurrently"""
        tasks = [self.process_query(query) for query in queries]
        results = await asyncio.gather(*tasks, return_exceptions=True)
        
        output = {}
        for result in results:
            if isinstance(result, Exception):
                logger.error(f"Error processing query: {result}")
            else:
                query, response = result
                output[query] = response
        
        return output


In [ ]:
async def main():
    
    
    # Initialize the system
    rag_system = RAGSystem(
        cohere_api_key=os.getenv("COHERE_API_KEY"),
        openrouter_api_key=os.getenv("OPENROUTER_API_KEY"),
        kb_folder="books",
        max_concurrent_requests=5
    )
    
    # Initialize the system
    await rag_system.initialize()
    
    
    queries = [
    "Who is Chris Olande?",
    "What has Chris Olande studied at Kenyatta University?",
    
    # Romeo and Juliet
    "What is the central conflict in Romeo and Juliet?",
    "How does Shakespeare portray love and fate in Romeo and Juliet?",
    "What role does Friar Laurence play in the tragedy of Romeo and Juliet?",
    "Compare the characters of Romeo and Tybalt",
    "How does Juliet's character evolve throughout the play?",

    # Declaration of Independence
    "What are the main ideas expressed in the Declaration of Independence?",
    "Who wrote the Declaration of Independence and why?",
    "How does the Declaration justify the colonies break from Britain?",
    "What Enlightenment principles are reflected in the Declaration?",
    "What impact did the Declaration of Independence have on global democratic movements?"
]
    
    # Process queries individually
    for query in queries:
        print(f"\n{'='*60}")
        print(f"Query: {query}")
        print('='*60)
        
        try:
            response = await rag_system.query(query)
            
            print(f"Needs Retrieval: {response.needs_retrieval}")
            print(f"Reasoning: {response.retrieval_decision_reasoning}")
            print(f"Retrieved Documents: {len(response.retrieved_docs)}")
            print(f"Reflection Score: {response.reflection_score}/10")
            
            if response.citations:
                print(f"Citations: {response.citations}")
            
            print(f"\nAnswer:\n{response.answer}")
            
        except Exception as e:
            print(f"Error processing query: {e}")
    
    
    print(f"\n{'='*60}")
    print("BATCH PROCESSING EXAMPLE")
    print('='*60)
    
    batch_processor = BatchProcessor(rag_system, max_concurrent=5)
    batch_queries = queries[:3]  # Process first 3 queries in batch
    
    batch_results = await batch_processor.process_batch(batch_queries)
    
    for query, response in batch_results.items():
        print(f"\nBatch Query: {query}")
        print(f"Reflection Score: {response.reflection_score}/10")
        print(f"Answer Length: {len(response.answer)} characters")

if __name__ == "__main__":
    await main()

INFO:__main__:Initializing the RAG system ...
INFO:__main__:Loaded 3 documents from books
INFO:__main__:Split 3 documents into 170 chunks


INFO:httpx:HTTP Request: POST https://api.cohere.com/v1/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.cohere.com/v1/embed "HTTP/1.1 200 OK"
INFO:faiss.loader:Loading faiss.
INFO:faiss.loader:Successfully loaded faiss.
INFO:faiss:Failed to load GPU Faiss: name 'GpuIndexIVFFlat' is not defined. Will not load constructor refs for GPU indexes. This is only an error if you're trying to use GPU Faiss.
INFO:__main__:Vector store saved to books/vector_store
INFO:__main__:Vector store automatically saved to books/vector_store
INFO:__main__:Self-RAG system initialized successfully



Query: Who is Chris Olande?


INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.cohere.com/v1/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"


Needs Retrieval: True
Reasoning: The query "Who is Chris Olande?" is asking for specific information about an individual, which suggests it requires factual data. This type of question typically seeks biographical information, which may not be universally known or could be about a less public figure, thus potentially requiring access to external documents or a knowledge base to provide an accurate answer. The answer would indeed benefit from external sources, as the information might not be common knowledge or could be subject to change over time, necessitating the most current or detailed data available. Given that the query is about a specific person and not general knowledge that everyone would know, and considering the possibility that Chris Olande might not be a widely recognized public figure, the query likely requires recent or specialized information that would be found in external documents or databases.
Retrieved Documents: 5
Reflection Score: 9.0/10
Citations: ['books/olande

...

INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.cohere.com/v1/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"


Needs Retrieval: True
Reasoning: The query "What are the main ideas expressed in the Declaration of Independence?" asks for specific information about a historical document. To answer this question, one would need to have knowledge of the document's content, which is typically learned through reading the document itself or studying its history.
Retrieved Documents: 5
Reflection Score: 9.0/10
Citations: ['books/declaration_of_independence_of_the_united_states.txt']

Answer:
The main ideas expressed in the Declaration of Independence, as outlined in the provided context documents, particularly in "declaration_of_independence_of_the_united_states.txt," can be summarized as follows:

1. **Necessity of Separation**: The document begins by stating that when a people find it necessary to dissolve the political bonds connecting them to another, they should declare the causes that lead to this separation (Document: declaration_of_independence_of_the_united_states.txt). This sets the stage for t

INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.cohere.com/v1/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"


Needs Retrieval: True
Reasoning: The query is specific, requires domain-specific information, and benefits from external documents or a knowledge base for a comprehensive answer. While general knowledge might provide a basic answer, detailed and accurate information, especially regarding the motivations behind the Declaration, necessitates external retrieval.
Retrieved Documents: 5
Reflection Score: 9.0/10
Citations: ['books/declaration_of_independence_of_the_united_states.txt']

Answer:
The Declaration of Independence was written by Thomas Jefferson, as stated in the document "The Project Gutenberg eBook of The Declaration of Independence of the United States of America" (declaration_of_independence_of_the_united_states.txt). This is explicitly mentioned in the section that provides information about the eBook, where it says "Author: Thomas Jefferson".

As for why the Declaration of Independence was written, the document itself provides insight into the motivations behind its creation

INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.cohere.com/v1/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
